In [1]:
import os
import g_requests
import pog_tree
os.getcwd()

'/home/seb-porras/GRASP_API/python_structures'

In [2]:
g_requests.requestJointReconstruction(aln="../test_data/big_test_data/GRASPTutorial_Final.aln",
                                      nwk="../test_data/big_test_data/GRASPTutorial_Final.nwk")

Socket created...

Connecting to server...

Socket connected to 10.139.1.21 on IP 4072

Closing socket...

{"Message":"Queued","Job":19}



'{"Message":"Queued","Job":19}\n'

In [2]:
extant_tree = g_requests.requestPOGTree(aln="../test_data/big_test_data/GRASPTutorial_Final.aln",
nwk="../test_data/big_test_data/GRASPTutorial_Final.nwk")

print('completed')

Socket created...

Connecting to server...

Socket connected to 10.139.1.21 on IP 4072

Closing socket...

completed


In [3]:
graphs = g_requests.requestJobOutput(19)


Socket created...

Connecting to server...

Socket connected to 10.139.1.21 on IP 4072

Closing socket...



In [4]:
tree = pog_tree.POGTreeFromJSON(extant_tree, graphs)
